In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
import copy as _copy
import pickle
import gzip
import os.path
import importlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import scipy.optimize
import functools

<h1> Developing fitting the model to response matrix measurement </h1>

The data have been treated and are available. Fits have been made to the different measureemnts. These revealed that the measurements can be explained with sufficient accuracy using second order polynoms. These fits are, however, model agnositic. 

The second step now aims to reach a model that fits the model to the measurement data

In this file different fit procedures are studied:
* Estimating the beta function from the model
* Fitting a second order scaling polynomial for the model
* Calculating reference data for each measurement assuming that the magnet transfer functions is not totally linear but contains a second order polynomial


In [ ]:
datetime.datetime.now().strftime('%Y %m %d')

<h2> Used modules </h2>

The different modules are collected in :mod:`bact2`.

* The response matrix directory collects the different module
* :mod:`reference_orbit` try to provide an *side effect free* interface to 
  ocelot next to orbit difference processing
* :mod:`commons` provides access to the preprocessed data. The current solution is a hack and will have to be adapted to the available data bases

* the preprocessed data are currently stored in a pickle file. The pickle file is created using 
`from_json_to_pickle`

In [ ]:
import bact2.applib.transverse_lib.bpm_data
importlib.reload(bact2.applib.transverse_lib.bpm_data)


In [ ]:
from bact2.applib.transverse_lib import reference_orbit
importlib.reload(reference_orbit)

In [ ]:
import bact2.applib.response_matrix.commons as commons
commons.pickle_file_name()

In [ ]:
path = os.getcwd()
path = os.path.join(path, *([os.path.pardir]*4 + ['data', 'response_matrix']) )
data_dir = os.path.normpath(path)

In [ ]:
pickle_file_name = 'preprocessed_steerer_response_data_tmp.bk.gz'
pickle_file_name = os.path.join(data_dir, pickle_file_name)

In [ ]:
with gzip.open(pickle_file_name) as fp:
    obj = pickle.load(fp)

In [ ]:
obj.original_dataframe.head()

In [ ]:
ttime = obj.original_dataframe.time / 1e3 / 3600

In [ ]:
[
    datetime.datetime.fromtimestamp(timestamp / 1e3) 
    for timestamp in (
        obj.original_dataframe.time.min(),
        obj.original_dataframe.time.max()
    )
]

In [ ]:
(ttime - ttime.min()).plot()

In [ ]:
importlib.reload(reference_orbit)

<h2> The model </h2>
The model is currently wrapped in :mod:`reference_orbit`. It allows
* calculating the reference orbit
* creating a new model with a changed element
* calculating offset from a changed orbit to the reference orbit without messing with the original model...

The following lines are used to set up the orbit and to store the reference data

In [ ]:
orbit = reference_orbit.OrbitCalculator(cell=reference_orbit.ncell)

In [ ]:
orbit_data_ref = orbit.orbitData()

In [ ]:
orbit_offset_filter = reference_orbit.OrbitOffset()
orbit_offset_filter.reference_data = orbit_data_ref

<h2> Developing the fit </h2>

The relevant data are stored in the processed dataframe of the pickle instance.
Please note that the measurement data stores the *power converter* variable name while the model 
refers to the magnet name. 

The function :func:`ps2magnet` converters a steerer power converter name to a magnet name. 

**NB**: the naming convention follows a location naming convention.

The model data are currently scaled: the bpm readings are in *mm* while the model data uses SI units and thus *m*.

In [ ]:

pc_magnet_info = pd.read_csv(
    os.path.join('/home/mfp/Devel/bessy_ii', 'ps-magnets.dbd'),
    names = ['power_converter', 'magnet', 'machine',
         'nominal_current', 'min_current', 'max_current', 
         'transfer_function'],
    delimiter=' ',
    header=None,  index_col=False
)
pc_magnet_info.head()

In [ ]:
model_to_bpm = 1000

In [ ]:
df = obj.processed_dataframe

In [ ]:
def ps2magnet(name):
    '''
    
    Todo: 
        add thorough checks!
    '''
    index = 3
    assert(name[index] == 'p')
    magnet_name = name[:index] + 'm' + name[index+1:]
    return magnet_name.upper()

def ps2magnet(name):
    name = name.upper()
    t_row = pc_magnet_info.loc[pc_magnet_info.power_converter == name,:]
    if t_row.shape[0] != 1:
        raise AssertionError(f'Found these {t_row} rows for supply {name}')
    magnet_name = t_row.magnet.values[0]
    return magnet_name

def magnet_transfer_function(magnet_name):
    t_row = pc_magnet_info.loc[pc_magnet_info.magnet == magnet_name,:]
    if t_row.shape[0] != 1:
        raise AssertionError(f'Found these {t_row} for magnet {magnet_name}')
    tf = t_row.transfer_function.values[0]
    return tf

    

Currently there is one steerer more working than available in my model

In [ ]:
def proces_bpm_data(ds, dx):
    '''
    
    remove the missing bpm at position 165
    '''
    bpms_in_model = np.absolute(ds - 165) > 1e-5
    dx_m = dx[bpms_in_model]
    ds_m = ds[bpms_in_model]
    return ds_m, dx_m

In [ ]:
def proces_bpm_data2D(ds, dx):
    '''
    
    remove the missing bpm at position 165
    
    Keep first axis
    '''       
    # n_rows = ds.shape[0]
    # check = dx.shape[0]
    #
    # if n_rows != check:
    #     txt = f'ds had {n_rows} rows but dx only {check}. Both must be equal'
    #     raise AssertionError(txt)
    
    ds_ref = ds[0]
    bpms_in_model = np.absolute(ds_ref - 165) > 1e-5
    
    dx_m = dx[:, bpms_in_model]
    ds_m = ds[:, bpms_in_model]
    
    return ds_m, dx_m

<h3> Selected steerer magnet  </h3>

Cuurrently only working with a single magnet. As soon it has been tested here a script will be developed for batch proessing all magnets

In [ ]:
ps_name =  'hs4p2d8r'
# ps_name =  'hs4p2d2r'


# ps_name = 'hs4p1d1r'
ps_name = 'hs1pd8r'

# ps_name = 'vs3p2d2r'
# ps_name = 'hs4p1d3r'

magnet_name = ps2magnet(ps_name)
transfer_function = magnet_transfer_function(magnet_name)
magnet_name, transfer_function

In [ ]:
180/np.pi

In [ ]:
df.columns

In [ ]:
df_sel = df.loc[
    (df.sc_selected == ps_name) 
    #& (df.ramp_index.isin([0, 4, 12, 16]))
    ,
    :]

In [ ]:
np.product(
    *pc_magnet_info.loc[pc_magnet_info.magnet == 'BM1D1R', ['nominal_current', 'transfer_function']].values
)

Currently starting with micro radians steering effects. Here the known magnet transfer functions should
be added. 

As a first check lets see what it can do ...

In [ ]:
0.855 / (2 * np.pi / 32)

# transfer_function * max_current * magnet_length / (b_main * rho) * 1000

In [ ]:
0.152 + 0.2755 + 0.4275

In [ ]:
def kicker_angle(magnet_name, current=None):
    '''
    
    Returns:
        angle in mrad
    '''
    magnet_length = 0.16
    
    row = pc_magnet_info.loc[pc_magnet_info.magnet == magnet_name, ['transfer_function', 'max_current']]
    assert(row.shape[0] == 1)

    transfer_function, max_current = row.values.T
    if current is None:
        current = max_current
    
    b_main = 1.3042
    rho = 4.3545
    
    bdl = transfer_function * current * magnet_length
    # Magnet length would have to be fit to the available power
    # 2 * pi had to be added to:
    #  * the devisor as b * rho * 2 pi give the total magnet length
    #  * the nominator as 2 * pi give a full circle
    angle = bdl / (b_main * rho)
    
    angle = float(angle)

    return angle

Calculate the maximum kicker angle for the selected magnet name in mrad

In [ ]:
kicker_angle(magnet_name) * 1000

The steerer can do roughly half a mrad

In [ ]:
t_angle = kicker_angle(magnet_name, current=df_sel.bk_dev_dI.max())
t_angle

In [ ]:
# t_angle = 1 * 10e-6
orbit_dev = orbit.orbitCalculatorForChangedMagnet(name=magnet_name, angle=t_angle)
orbit_dev.getElementbyName(magnet_name)

The following lines calculate the deviated orbit and its offset to the reference orbit. These data 
are used for starting the fit.

In [ ]:
dev_data = orbit_dev.orbitData()

In [ ]:
offset_data = orbit_offset_filter(dev_data)

<h3> A simple scale fit using selected data </h3>

This was the very first try to see that the correct element is selected when the deviated orbit is used. 
The non linear least squares fit is used as I did not yet find a suitable flexible linear fit in scipy (e.g. GSL's multifit)

In [ ]:
# is there a multfit like in gsl? did not find one ...

def min_func(params, *args, bpm_data = None, model_data = None):
    scale, = params
    
    assert(bpm_data is not None)
    assert(model_data is not None)
    
    dval = bpm_data - model_data * scale
    #check = np.absolute(dval)
    #print(check.max())
    return dval

The jacobian can be computed numerically. It is required further down. Thus it is considered wise to 
get practice here on the simple model.

In [ ]:
def jac(params, *args, bpm_data = None, model_data = None):
    scale, = params
    
    assert(model_data is not None)
    dval = -model_data
    return dval[:, np.newaxis]

In [ ]:

bpm_fit_ref_data = df_sel.iloc[0, :]
bpm_fit_data = df_sel.iloc[4, :]

if True:
    dx = bpm_fit_data.bpm_waveform_x_pos - bpm_fit_ref_data.bpm_waveform_x_pos
    ds_m, dx_m  = proces_bpm_data(bpm_fit_data.bpm_waveform_ds, dx)
    d = {
        'model_data': offset_data.bpm.x * 1000,
        'bpm_data': dx_m
    }
else: 
    dy = bpm_fit_data.bpm_waveform_y_pos - bpm_fit_ref_data.bpm_waveform_y_pos
    ds_m, dy_m  = proces_bpm_data(bpm_fit_data.bpm_waveform_ds, dy)
    d = {
        'model_data': offset_data.bpm.y * 1000,
        'bpm_data': dy_m
    }

res = scipy.optimize.least_squares(min_func, x0=(1.,), kwargs=d)
scale_model = float(res.x)

In [ ]:
assert(res.success)
res.message

In [ ]:
assert(res.success)
scale_model, scale_model/ (2 * np.pi)

A scale in the range of 10 is typical. Currently it seems that the polarity of the steerers is *negative* by default. A value of excatly one is suspicious

<h3> An individual scaling fit to each measurement </h3> 

This reflects the development. Again it was useful to get an idea if a linear fit can describe the data. 

1. The first plot below shows:
   * the bpm measured data: here only the difference to the reference orbit is shown. Currently the reference orbit is the one of the first measurement. The bpm data are marked with crosses.
   * the scaled model data: these data are scaled linearly for each individual measurement. 
2. The second plot below shows:
   * the offset of the bpm data from the model in absolute numbers (left y axis)
3. The third plot below shows:
   * the relative value of the offset: thus the offset devided by the relative current excitation

One can see that the model seems to work. Some BPM's are siginficantly off the model. 
The model agnostic polynomial BPM/magnet fits revealed that the BPM's can measure with an precision of $\approx$ 1 $\mu$m. Furthermore one could see that hysteresis effects are below 50 $\mu$m. Thus the measurements below have a SNR of at least 10 or more.


In a first step the model is adjusted to the found scale.

In [ ]:
t_updated_angle = t_angle * scale_model
orbit_dev_updated = orbit.orbitCalculatorForChangedMagnet(name=magnet_name, angle=t_updated_angle)
orbit_dev_updated.getElementbyName(magnet_name)

The following lines calculate the deviated orbit and its offset to the reference orbit. These data 
are used for starting the fit.

In [ ]:
dev_data_updated = orbit_dev_updated.orbitData()

In [ ]:
offset_data_updated = orbit_offset_filter(dev_data_updated)

The fits below are expexted to give a scale close to 1

In [ ]:
fig = plt.figure(figsize=[20,24])
ax1 = plt.subplot(311)
# BPM at ds = 165 is not in the model
ax2 = plt.subplot(312)
ax3 = plt.subplot(313)
#ax3 = ax2.twinx()


for ax in [ax1, ax2, ax3]:
    ax.clear()
    del ax

dI_max = df_sel.bk_dev_dI.abs().max()

ref_x = None
for cnt in range(len(df_sel.bpm_x_ref)):
    if cnt > 1:
        # break
        pass
    
    row = df_sel.iloc[cnt, :]
    x = row.bpm_waveform_x_pos
    y = row.bpm_waveform_y_pos
    dI = row.bk_dev_dI
    dI_s = dI / dI_max
    ds = row.bpm_waveform_ds
    if ref_x is None:
        ref_x = x
    dx = x - ref_x


    # A fit is only sensible if sufficient offset is available
    adx = np.absolute(dx)
    check = adx.max()
    threshold = 0.1
    
    do_fit = True
    if check < threshold:
        print(f'No fit for cnt {cnt}')
        do_fit = False
        p_scale = 1
        
    
    if do_fit:
        # Prepare for fit
        ds_m, dx_m = proces_bpm_data(ds, dx)
        d = {
               'model_data': offset_data_updated.bpm.x * model_to_bpm,
               'bpm_data': dx_m
        }
    
        scale_model_corr = None
        res = scipy.optimize.least_squares(min_func, jac=jac, x0=(1.,), kwargs=d)
        scale_model_corr = float(res.x)
        dI = row.bk_dev_dI
        factor = scale_model_corr / dI_s
        rel_factor = (factor -1) * 100
        print(
            f'steerer current {dI:.3f}, scale = {scale_model_corr:.3f}, factor = {factor:.3f}'
            f' factor deviation = {rel_factor:.2f}%'
        )
        
        if -scale_model_corr < 0:
            p_scale = -1
        else:
            p_scale = 1
            

    if do_fit:        
        scale_model_current = (dI_max / dI)
        marker = '+'
        linestyle = '--'
    else:
        scale_model_current = 1
        marker = '.'
        linestyle = ':'
        
    # BPM measurement data
    
    # if scaling with model current p_scale does not make sense
    p_scale = 1
    # A line helping the eye
    pdx   = dx * p_scale * scale_model_current
    pdx_m = dx_m * p_scale * scale_model_current
    
    line, = ax1.plot(ds, pdx, marker=marker)
    lcol = color=line.get_color()
    ax1.plot(ds, pdx, linestyle=linestyle, color=lcol, linewidth=.25)
        
    
    if do_fit:
        
        bpm_x_mod = offset_data.bpm.x *  model_to_bpm * scale_model * scale_model_corr
        pbpm_x_mod = bpm_x_mod * scale_model_current
        
        ax1.plot(offset_data.bpm.s,   pbpm_x_mod * p_scale, '-',  color=lcol, linewidth=.25)
        ax1.plot(offset_data.bpm.s,   pbpm_x_mod * p_scale, '+', color=lcol)

        
        delta_x = (dx_m - bpm_x_mod) * scale_model_current
        # Convenient to disply model separately
        # delta_x = bpm_x_mod * scale_model_current
        # delta_x = pdx_m
        ax2.plot(ds_m, delta_x * p_scale, '+-', color=lcol, linewidth=.25)
        
        scale_threshold = .05
        idx = np.absolute(bpm_x_mod) > scale_threshold
        ds_m2 = ds_m[idx]
        sf = delta_x[idx] / bpm_x_mod[idx] * scale_model_current

       
        ax3.plot(ds_m2, sf * p_scale, '+--', color=lcol, linewidth=.25)

ax1.set_xlabel('ds [m]')
ax2.set_xlabel('ds [m]')
ax1.set_ylabel('x [mm]')
ax2.set_ylabel('$\Delta$ x [mm]')
ax3.set_ylabel('rx')

<h3> Fitting the steerer transfer function: simple approach </h3>

The steerer transfer function -- i.e the angle it makes per Ampere -- is assumed to be a polynomial of second order.

Results show that an offset can be hardly see for the magnets tested up to now. The plots follow the same convention as above.

In [ ]:
def to_parameters(params, simple=True):
    if not simple:
        return params
    
    slope, = params
    slope = float(slope)
    intercept = 0
    parabola = 0
    
    return  intercept, slope, parabola

def model_array_from_data(data):
    '''
    
    Let the computer check that jac and func are compatible
    
    Made over array sizes currently
    '''
    return data[np.newaxis, :]

def compute_scaling(dI, intercept, slope, parabola):
    scale = intercept + dI * (slope + dI * parabola)
    return scale 

def compute_reference_data_all(params, *args, simple=True, dI=None, model_data=None):
    
    assert(dI is not None)
    assert(model_data is not None)
    
    dI = np.atleast_1d(dI)
    t_pars = to_parameters(params, simple=simple)
    scale = compute_scaling(dI, *t_pars)
    md = model_array_from_data(model_data)
    ref =  md * scale[:, np.newaxis]

    return ref

def min_func_all(params, *args, bpm_data= None, **kws):

    assert(bpm_data is not None)
    
    ref = compute_reference_data_all(params, *args, **kws)
    dval = bpm_data - ref
    
    dval = np.ravel(dval)
    return dval
    

def jac_all(params, *args, simple=True, dI=None, model_data=None, bpm_data=None):
    
    dI = np.atleast_1d(dI)
    intercept, slope, parabola = to_parameters(params, simple=simple)
    # How many parameters 
    if simple:
        m = 1
    else:
        m = 3   
    
    # Some helper arrays with appropriate extra dimensions
    # These should be in the same order as for the function
    # Then the ravel below will not mess around
    md = model_array_from_data(model_data)
    dIa = dI[:, np.newaxis]
    ones = np.ones(dIa.shape)
    
    # The contribution of the linear term ...
    # i.e. the slope factor
    linear = md * dIa
    
    linear_contrib = np.ravel(linear)
    
    if not simple:
        const = md * ones
        quadratic = md * (dIa)**2
        
        const_contrib = np.ravel(const)
        quadratic_contrib = np.ravel(quadratic)
        
    jac = np.zeros([linear_contrib.shape[0], m], np.float_)

    if simple:
        jac[:, 0] = linear_contrib
    else:
        jac[:, 0] = const_contrib
        jac[:, 1] = linear_contrib
        jac[:, 2] = quadratic_contrib
        
    # measurement data - model
    jac = -jac
    return jac

In [ ]:
ref = df_sel.iloc[0, :]
x0  = ref.bpm_waveform_x_pos
y0  = ref.bpm_waveform_y_pos

x  = df_sel.bpm_waveform_x_pos
y  = df_sel.bpm_waveform_y_pos

dI = df_sel.bk_dev_dI
dI_max = dI.abs().max()
dI_s = dI/dI_max

ds = df_sel.bpm_waveform_ds

dx = x - x0
dy = y - y0

dx = np.array(dx.tolist(), np.float_)
dy = np.array(dy.tolist(), np.float_)
ds = np.array(ds.tolist(), np.float_)
ds_m, dx_m = proces_bpm_data2D(ds, dx)

simple = False
d_model = {
    'model_data': offset_data_updated.bpm.x * model_to_bpm,
    'dI' : dI_s,
    'simple' : simple
}

d = _copy.copy(d_model)
d['bpm_data'] = dx_m

if simple: 
    res = scipy.optimize.least_squares(min_func_all, x0=(0), kwargs=d)
    intecept = 0
    slope = float(res.x)
    parabola = 0
    
else:
    res_fit = scipy.optimize.least_squares(min_func_all, jac=jac_all, x0=(0, 1, 0), kwargs=d)
    intercept, slope, parabola = res_fit.x

ref_data = compute_reference_data_all(res_fit.x, **d_model)
for ax in [ax1, ax2, ax3]:
    ax.clear()
    del ax


fig = plt.figure(figsize=[20, 24])
ax1 = plt.subplot(311)
ax2 = plt.subplot(312)
ax3 = plt.subplot(313)
# ax3 = ax2.twinx()

# a polarity scale
p_scale = 1
for row in range(ref_data.shape[0]):
    # BPM measurement data
    # A line helping the eye
    dx_row = dx[row, :]
    ds_row = ds[row, :]
    dx_ref_row = ref_data[row, :]
    
    dx_m_row = dx_m[row, :]
    
    dI_s_row = dI_s.iloc[row]
    check = np.absolute(dI_s_row)
    _eps = 1e-5
    if check < _eps:
        scale_plot_by_current = 1
    else:
        scale_plot_by_current = 1./dI_s_row
        
    pdx_row = dx_row * p_scale
    pdx_m_row = dx_m_row * p_scale
    pdx_ref_row = dx_ref_row * p_scale
    
    pdx_row     = pdx_row * scale_plot_by_current
    pdx_mrow    = pdx_m_row * scale_plot_by_current
    pdx_ref_row = pdx_ref_row * scale_plot_by_current
    # The measured data 
    line, = ax1.plot(ds_row, pdx_row, 'x')
    lcol = color=line.get_color()    
    ax1.plot(ds_row, pdx_row, '-', color=lcol, linewidth=.25)
    
    # The fitted ones 
    ax1.plot(offset_data.bpm.s,   pdx_ref_row, '+-', color=lcol, linewidth=.25)
  
    # Plotting the offset from the fit to all other data
    offset = dx_m_row - dx_ref_row
    poffset = offset * p_scale 
    poffset = poffset * scale_plot_by_current
    ax2.plot(offset_data.bpm.s, poffset, '+-', color=lcol, linewidth=.25)
    # relative difference only if significant measurement data
    # currently for bpm measurements above 0.05 mm
    bpm_min_measurement = 0.05 
    idx = np.absolute(pdx_m_row) > bpm_min_measurement
    
    offset_sel = offset[idx]
    ds_sel = offset_data.bpm.s[idx]
    dx_ref_row_sel = dx_ref_row[idx]
    dx_rel = offset_sel / dx_ref_row_sel
    
    ax3.plot(ds_sel, dx_rel, '.--', color=lcol, linewidth=.25)

ax1.set_xlabel('ds [m]')
ax2.set_xlabel('ds [m]')
ax1.set_ylabel('x [mm]')
ax2.set_ylabel('$\Delta$ x [mm]')
ax3.set_ylabel('rx')
res_fit.x

In [ ]:
res_fit.nfev, res_fit.njev

<h2> Fitting the model to the data using model data for each current </h2>

Evaluating ocelot at each step is a bit slow. So here now reference data are computed
for each set of data. The fit above gave rather small deviations. Therefore one can 
assume that an approximation should be sufficient.

In [ ]:
t_updated_angle

In [ ]:
@functools.lru_cache(maxsize=None)
def compute_reference_model(scale, t_magnet_name=None):
    
    assert(t_magnet_name is not None)
    
    scaled_angle = t_updated_angle * scale
    print(f'Computing model for scale {scaled_angle*1000:.3f} mrad and magnet {t_magnet_name}')
    
    orbit_dev_ = orbit.orbitCalculatorForChangedMagnet(name=t_magnet_name, angle=scaled_angle)
    orbit_dev_.getElementbyName(magnet_name)
    dev_data_ = orbit_dev_.orbitData()
    offset_data_= orbit_offset_filter(dev_data_)
    
    return offset_data_

Construct the model data:
 1. First compute the model data
 2. scale them all to the reference scaled current 1.0 (otherwise the used interpolation does not 
    work any more)
 3. repeat the last step once more just to see if the little scaling still has an impact
 

In [ ]:
def compute_and_scale(scale):
    r = compute_reference_model(scale, t_magnet_name=magnet_name)
    dx = r.bpm.x
    return dx/scale

In [ ]:
dI = df_sel.bk_dev_dI
dI_max = dI.abs().max()
dI_s = dI/dI_max

scales = compute_scaling(dI_s, *res_fit.x)

model_data = np.array([compute_and_scale(scale) for scale in scales])
model_data.shape

In [ ]:
def compute_reference_data_adjust(params, *args, simple=True, dI=None, model_data=None):
    
    assert(dI is not None)
    assert(model_data is not None)
    
    dI = np.atleast_1d(dI)
    t_pars = to_parameters(params, simple=simple)
    scale = compute_scaling(dI, *t_pars)
    md = model_data
    ref =  md * scale[:, np.newaxis]

    return ref

def min_func_adjust(params, *args, bpm_data= None, **kws):

    assert(bpm_data is not None)
    
    ref = compute_reference_data_adjust(params, *args, **kws)
    dval = bpm_data - ref
    
    dval = np.ravel(dval)
    return dval
    

def jac_adjust(params, *args, simple=True, dI=None, model_data=None, bpm_data=None):
    
    dI = np.atleast_1d(dI)
    intercept, slope, parabola = to_parameters(params, simple=simple)
    # How many parameters 
    if simple:
        m = 1
    else:
        m = 3   
    
    # Some helper arrays with appropriate extra dimensions
    # These should be in the same order as for the function
    # Then the ravel below will not mess around
    md = model_data
    dIa = dI[:, np.newaxis]
    ones = np.ones(dIa.shape)
    
    # The contribution of the linear term ...
    # i.e. the slope factor
    linear = md * dIa
    
    linear_contrib = np.ravel(linear)
    
    if not simple:
        const = md * ones
        quadratic = md * (dIa)**2
        
        const_contrib = np.ravel(const)
        quadratic_contrib = np.ravel(quadratic)
        
    jac = np.zeros([linear_contrib.shape[0], m], np.float_)

    if simple:
        jac[:, 0] = linear_contrib
    else:
        jac[:, 0] = const_contrib
        jac[:, 1] = linear_contrib
        jac[:, 2] = quadratic_contrib
        
    # measurement data - model
    jac = -jac
    return jac

In [ ]:
ref = df_sel.iloc[0, :]
x0  = ref.bpm_waveform_x_pos
y0  = ref.bpm_waveform_y_pos

x  = df_sel.bpm_waveform_x_pos
y  = df_sel.bpm_waveform_y_pos

dI = df_sel.bk_dev_dI
dI_max = dI.abs().max()
dI_s = dI/dI_max

ds = df_sel.bpm_waveform_ds

dx = x - x0
dy = y - y0

dx = np.array(dx.tolist(), np.float_)
dy = np.array(dy.tolist(), np.float_)
ds = np.array(ds.tolist(), np.float_)
ds_m, dx_m = proces_bpm_data2D(ds, dx)

simple = False


assert(simple == False)
for i in range(2):
    if i == 0:
        t_par = res_fit.x
    else:
        t_par = res_adjust.x
    scales = compute_scaling(dI_s, *t_par)
    model_data = np.array([compute_and_scale(scale) for scale in scales])
    
    d_model = {
        'model_data': model_data * model_to_bpm,
        'dI' : dI_s,
        'simple' : simple
    }

    d = _copy.copy(d_model)
    d['bpm_data'] = dx_m
    res_adjust = scipy.optimize.least_squares(min_func_adjust, x0=t_par, kwargs=d)

res_adjust

In [ ]:
ref_data = compute_reference_data_adjust(res_adjust.x, **d_model)
for ax in [ax1, ax2, ax3]:
    ax.clear()
    del ax


fig = plt.figure(figsize=[20,24])
ax1 = plt.subplot(311)
ax2 = plt.subplot(312)
ax3 = plt.subplot(313)
# ax3 = ax2.twinx()

# a polarity scale
p_scale = 1
for row in range(ref_data.shape[0]):
    # BPM measurement data
    # A line helping the eye
    dx_row = dx[row, :]
    ds_row = ds[row, :]
    
    dx_ref_row = ref_data[row, :]

    # dx_ref_row = model_data[row, :] * model_to_bpm
    
    dx_m_row = dx_m[row, :]
    
    pdx_row = dx_row * p_scale
    pdx_m_row = dx_m_row * p_scale
    pdx_ref_row = dx_ref_row * p_scale
    
    # The measured data 
    line, = ax1.plot(ds_row, pdx_row, 'x')
    lcol = color=line.get_color()    
    ax1.plot(ds_row, pdx_row, '-', color=lcol, linewidth=.25)
    
    # The fitted ones 
    ax1.plot(offset_data.bpm.s,   pdx_ref_row, '+-', color=lcol, linewidth=.25)
  
    # Plotting the offset from the fit to all other data
    offset = pdx_m_row - pdx_ref_row
    # offset = pdx_ref_row
    ax2.plot(offset_data.bpm.s, offset, '+-', color=lcol, linewidth=.25)
    # relative difference only if significant measurement data
    # currently for bpm measurements above 0.05 mm
    bpm_min_measurement = 0.05 
    idx = np.absolute(pdx_m_row) > bpm_min_measurement
    
    offset_sel = offset[idx]
    ds_sel = offset_data.bpm.s[idx]
    dx_ref_row_sel = dx_ref_row[idx]
    scale = offset_sel / dx_ref_row_sel
    
    ax3.plot(ds_sel, scale, '.--', color=lcol, linewidth=.25)

ax1.set_xlabel('ds [m]')
ax2.set_xlabel('ds [m]')
ax1.set_ylabel('x [mm]')
ax2.set_ylabel('$\Delta$ x [mm]')
ax3.set_ylabel('rx')

In [ ]:
res_adjust.nfev, res_adjust.njev

In [ ]:
res_adjust.x, res_fit.x

In [ ]:
res_adjust.x - res_fit.x

<h3> Fitting model data in x and y at once </h3>

Measurements show that the steerer also influence the other axis. Let's get prepared to investigate this effect

In [ ]:
def check_steerer_name(t_magnet_name):
    steerer_type = t_magnet_name[0]        
    if steerer_type not in 'HV':
        txt = f'Steerer type {steerer_type} with name {t_magnet_name} neither horizontal nor vertical'
        raise AssertionError(txt)

    return steerer_type

def corresponding_steerer(t_magnet_name):
    steerer_type = check_steerer_name(t_magnet_name)
    if steerer_type == 'H':
        suffix = '_artefact'
    elif steerer_type == 'V':
        suffix = '_artefact'
    else:
        assert(0)
        
    r = t_magnet_name + suffix
    return r
    
corresponding_steerer(magnet_name)

In [ ]:
@functools.lru_cache(maxsize=None)
def compute_reference_model_xy(x_magnet_name=None, y_magnet_name=None, x_scale=None, y_scale=None):
    
    assert(x_magnet_name is not None)
    assert(y_magnet_name is not None)
    assert(x_scale is not None)
    assert(y_scale is not None)
    
    x_scaled_angle = t_updated_angle * x_scale
    y_scaled_angle = t_updated_angle * y_scale
    print(
        f'Computing model for magnets'
        f' x {x_magnet_name} '
        f' y {y_magnet_name} '
        f' scale x = {x_scaled_angle*1000:.3f} mrad'
        f' scale y = {y_scaled_angle*1000:.3f} mrad'
    )
    
    orbit_dev_ = orbit.orbitCalculatorForChangedMagnet(name=x_magnet_name, angle=x_scaled_angle, init_lattice=False)
    orbit_dev_ = orbit_dev_.orbitCalculatorForChangedMagnet(name=y_magnet_name, angle=y_scaled_angle, init_lattice=True)
    dev_data_ = orbit_dev_.orbitData()
    offset_data_= orbit_offset_filter(dev_data_)
    
    return offset_data_

In [ ]:
def compute_and_scale_2D(scale):
    r = compute_reference_model(scale, t_magnet_name=magnet_name)
    dx = r.bpm.x
    dy = r.bpm.y
    dxs = dx/scale * model_to_bpm
    dys = dy/scale * model_to_bpm
    r = reference_orbit.OrbitData(x=dxs, y=dys, s=r.bpm.s)
    return r

In [ ]:
def compute_reference_data_adjust_2D(params, *args, simple=True, dI=None, model_data=None):
    ''' add an effect of xy
    '''
    assert(dI is not None)
    assert(model_data is not None)
    
    dI = np.atleast_1d(dI)
    t_pars = to_parameters(params, simple=simple)
    scale = compute_scaling(dI, *t_pars)

    mdx = model_data.x
    mdy = model_data.y
    ref_x =  mdx * scale[:, np.newaxis]
    ref_y =  mdy * scale[:, np.newaxis]

    ref = reference_orbit.OrbitData(x=ref_x, y=ref_y, s=model_data.s)
    return ref

def min_func_adjust_2D(params, *args, bpm_data= None, **kws):
    '''Minimize the radial offset
    '''
    assert(bpm_data is not None)
    
    ref = compute_reference_data_adjust_2D(params, *args, **kws)
    dval_x = bpm_data.x - ref.x
    dval_y = bpm_data.y - ref.y
    
    dval2 = dval_x ** 2 + dval_y **2
    dval = np.sqrt(dval2)
    dval = np.ravel(dval)
    return dval
    

def jac_adjust_2D(params, *args, simple=True, dI=None, model_data=None, bpm_data=None):
    
    raise NotImplementedError()
    
    dI = np.atleast_1d(dI)
    intercept, slope, parabola = to_parameters(params, simple=simple)
    # How many parameters 
    if simple:
        m = 1
    else:
        m = 3   
    
    # Some helper arrays with appropriate extra dimensions
    # These should be in the same order as for the function
    # Then the ravel below will not mess around
    md = model_data
    dIa = dI[:, np.newaxis]
    ones = np.ones(dIa.shape)
    
    # The contribution of the linear term ...
    # i.e. the slope factor
    linear = md * dIa
    
    linear_contrib = np.ravel(linear)
    
    if not simple:
        const = md * ones
        quadratic = md * (dIa)**2
        
        const_contrib = np.ravel(const)
        quadratic_contrib = np.ravel(quadratic)
        
    jac = np.zeros([linear_contrib.shape[0], m], np.float_)

    if simple:
        jac[:, 0] = linear_contrib
    else:
        jac[:, 0] = const_contrib
        jac[:, 1] = linear_contrib
        jac[:, 2] = quadratic_contrib
        
    # measurement data - model
    jac = -jac
    return jac

In [ ]:
def compute_reference_data_adjust_x_y(params, *args, simple=True, dI=None, model_data=None):
    ''' add an effect of xy
    '''
    assert(dI is not None)
    assert(model_data is not None)
    
    dI = np.atleast_1d(dI)
    t_pars = to_parameters(params[:-1], simple=simple)
    scale_other = params[-1]
    scale = compute_scaling(dI, *t_pars)

    mdx = model_data.x
    mdy = model_data.y
    ref_x =  mdx * scale[:, np.newaxis]
    ref_y =  mdy * scale[:, np.newaxis] * scale_other

    ref = reference_orbit.OrbitData(x=ref_x, y=ref_y, s=model_data.s)
    return ref

def min_func_adjust_x_y(params, *args, bpm_data= None, **kws):
    '''Minimize the radial offset
    '''
    assert(bpm_data is not None)
    
    ref = compute_reference_data_adjust_x_y(params, *args, **kws)
    dval_x = bpm_data.x - ref.x
    dval_y = bpm_data.y - ref.y
    
    dval2 = dval_x ** 2 + dval_y **2
    dval = np.sqrt(dval2)
    dval = np.ravel(dval)
    return dval
    


In [ ]:
def compute_and_scale_x_y(scale, scale_other):
    corresponding_steerer_name = corresponding_steerer(magnet_name)
    steerer_type = check_steerer_name(magnet_name)
    
    scales  = scale, scale_other
    steerers = magnet_name, corresponding_steerer_name
    
    if steerer_type == 'H':
        x_steerer, y_steerer = steerers
        x_scale, y_scale = scales
        
    elif steerer_type == 'V':
        y_steerer, x_steerer = steerers
        y_scale, x_scale = scales
    else:
        assert(0)
    
    r = compute_reference_model_xy(
        x_magnet_name=x_steerer,
        y_magnet_name=y_steerer,
        x_scale=x_scale,
        y_scale=y_scale)
    
    dx = r.bpm.x
    dy = r.bpm.y
    dxs = dx/scale * model_to_bpm
    dys = dy/scale * model_to_bpm
    r = reference_orbit.OrbitData(x=dxs, y=dys, s=r.bpm.s)
    return r

The following code decides if a double model is made or not

In [ ]:
xy_scale = False
xy_scale = True

In [ ]:


dI = df_sel.bk_dev_dI
dI_max = dI.abs().max()
dI_s = dI/dI_max

scales = compute_scaling(dI_s, *res_adjust.x)


def create_model_data(scales):
    data = [compute_and_scale_2D(scale) for scale in scales]
    data = reference_orbit.OrbitData(
        x = np.array([md.x for md in data]),
        y = np.array([md.y for md in data]),
        s = orbit_data_ref.trace.s
    )
    return data

def create_model_data_x_y(scales, scale_other):
    data = [compute_and_scale_x_y(scale, scale * scale_other) for scale in scales]
    data = reference_orbit.OrbitData(
        x = np.array([md.x for md in data]),
        y = np.array([md.y for md in data]),
        s = orbit_data_ref.trace.s
    )
    return data

if xy_scale:
    scale_other = .2
    model_data = create_model_data_x_y(scales, scale_other)        
else:
    model_data = create_model_data(scales)
    # just to have it defined
    # scale_other = 1


In [ ]:
ref = df_sel.iloc[0, :]
x0  = ref.bpm_waveform_x_pos
y0  = ref.bpm_waveform_y_pos

x  = df_sel.bpm_waveform_x_pos
y  = df_sel.bpm_waveform_y_pos

dI = df_sel.bk_dev_dI
dI_max = dI.abs().max()
dI_s = dI/dI_max

ds = df_sel.bpm_waveform_ds

dx = x - x0
dy = y - y0

dx = np.array(dx.tolist(), np.float_)
dy = np.array(dy.tolist(), np.float_)
ds = np.array(ds.tolist(), np.float_)
ds_m, dx_m = proces_bpm_data2D(ds, dx)
ds_m, dy_m = proces_bpm_data2D(ds, dy)

bpm_data = reference_orbit.OrbitData(x=dx, y=dy, s=ds)
bpm_data_m = reference_orbit.OrbitData(x=dx_m, y=dy_m, s=ds_m)
simple = False
    
assert(simple == False)
t_par = res_adjust.x

if xy_scale:
    scale_other_update = scale_other
    all_pars = t_par.tolist() + [scale_other_update]
    func = min_func_adjust_x_y
else:
    all_pars = t_par
    func = min_func_adjust_2D
    
for i in range(2):
    if i > 0:        
        all_pars = res_adjust2D.x
        if xy_scale:
            t_par = all_pars[:-1]
            scale_other_update = all_pars[-1]
        else:
            t_par = all_pars
    
    print(all_pars, t_par)
    scales = compute_scaling(dI_s, *t_par)
    
    if xy_scale:
        pass
    else:
        model_data = create_model_data(scales)
        
    d_model = {
        'model_data': model_data,
        'dI' : dI_s,
        'simple' : simple
    }

    d = _copy.copy(d_model)
    d['bpm_data'] = bpm_data_m

    res_adjust2D = scipy.optimize.least_squares(func, x0=all_pars, kwargs=d)

res_adjust2D

In [ ]:
_eps = 1e-6
prelative_current_scale = np.where(np.absolute(dI_s) < _eps, None, 1/dI_s)
prelative_current_scale

In [ ]:
if xy_scale:
    ref_data = compute_reference_data_adjust_x_y(res_adjust2D.x, **d_model)
else:
    ref_data = compute_reference_data_adjust_2D(res_adjust2D.x, **d_model)

for ax in [ax1, ax2, ax3]:
    ax.clear()
    del ax


fig = plt.figure(figsize=[20,24])
ax1 = plt.subplot(311)
ax2 = plt.subplot(312)
ax3 = plt.subplot(313)

# ax3 = ax2.twinx()

# a polarity scale
p_scale = 1
for row in range(ref_data.x.shape[0]):
    # BPM measurement data
    # A line helping the eye
    dx_row = bpm_data.x[row, :]
    ds_row = bpm_data.s[row, :]
    
    plot_scale = prelative_current_scale[row]
    if plot_scale is None:
        plot_scale = 1/8
    plot_scale = p_scale * plot_scale
        
    dx_ref_row = ref_data.x[row, :]

    # dx_ref_row = model_data[row, :] * model_to_bpm
    
    dx_m_row = bpm_data_m.x[row, :]
    
    pdx_row = dx_row  * plot_scale
    pdx_m_row = dx_m_row * plot_scale
    pdx_ref_row = dx_ref_row * plot_scale
    
    # The measured data 
    line, = ax1.plot(ds_row, pdx_row, 'x')
    lcol = color=line.get_color()    
    ax1.plot(ds_row, pdx_row, '-', color=lcol, linewidth=.25)
    
    # The fitted ones 
    ax1.plot(offset_data.bpm.s,   pdx_ref_row, '+-', color=lcol, linewidth=.25)
  
    # Plotting the offset from the fit to all other data
    offset = (dx_m_row - dx_ref_row)
    poffset = offset * plot_scale
    # offset = pdx_ref_row
    ax2.plot(offset_data.bpm.s, poffset, '+-', color=lcol, linewidth=.25)
    # relative difference only if significant measurement data
    # currently for bpm measurements above 0.05 mm
    bpm_min_measurement = 0.05 
    idx = np.absolute(pdx_m_row) > bpm_min_measurement
    
    offset_sel = offset[idx]
    ds_sel = offset_data.bpm.s[idx]
    dx_ref_row_sel = dx_ref_row[idx]
    scale = offset_sel / dx_ref_row_sel
    
    ax3.plot(ds_sel, scale, '.--', color=lcol, linewidth=.25)

ax1.set_xlabel('ds [m]')
ax2.set_xlabel('ds [m]')
ax1.set_ylabel('x [mm]')
ax2.set_ylabel('$\Delta$ x [mm]')
ax3.set_ylabel('rx')

In [ ]:
for ax in [ax1, ax2, ax3]:
    ax.clear()
    del ax


fig = plt.figure(figsize=[20,24])
ax1 = plt.subplot(311)
ax2 = plt.subplot(312)
ax3 = plt.subplot(313)

# ax3 = ax2.twinx()

# a polarity scale
p_scale = 1
for row in range(ref_data.x.shape[0]):
    # BPM measurement data
    # A line helping the eye
    dy_row = bpm_data.y[row, :]
    ds_row = bpm_data.s[row, :]
    
    dy_ref_row = ref_data.y[row, :] * 5

    # dx_ref_row = model_data[row, :] * model_to_bpm
    
    dy_m_row = bpm_data_m.y[row, :]
    
    plot_scale = prelative_current_scale[row]
    if plot_scale is None:
        plot_scale = 1/8
    plot_scale = p_scale * plot_scale
    
    pdy_row = dy_row * plot_scale
    pdy_m_row = dy_m_row * plot_scale
    pdy_ref_row = dy_ref_row * plot_scale
    
    # The measured data 
    line, = ax1.plot(ds_row, pdy_row, 'x')
    lcol = color=line.get_color()    
    ax1.plot(ds_row, pdy_row, '-', color=lcol, linewidth=.25)
    
    # The fitted ones 
    ax1.plot(offset_data.bpm.s,   pdy_ref_row, '+-', color=lcol, linewidth=.25)
  
    # Plotting the offset from the fit to all other data
    offset = dy_m_row - dy_ref_row
    # offset = dy_ref_row
    poffset = offset * plot_scale
    ax2.plot(offset_data.bpm.s, poffset, '+-', color=lcol, linewidth=.25)
    # relative difference only if significant measurement data
    # currently for bpm measurements above 0.05 mm
    bpm_min_measurement = 0.05 
    idx = np.absolute(pdy_m_row) > bpm_min_measurement
    
    offset_sel = offset[idx]
    dy_sel = offset_data.bpm.s[idx]
    dy_ref_row_sel = dy_ref_row[idx]
    scale = offset_sel / dy_ref_row_sel
    
    #ax3.plot(ds_sel, scale, '.--', color=lcol, linewidth=.25)

ax1.set_xlabel('ds [m]')
ax2.set_xlabel('ds [m]')
ax1.set_ylabel('y [mm]')
# ax2.set_ylabel('$\Delta$ y [mm]')
ax2.set_ylabel('$y_{ref}$ [mm]')
ax3.set_ylabel('ry')

In [ ]:
twiss = orbit.twissParameters()
twiss_bpms = orbit.twissParametersBpms()

In [ ]:
beta_x, beta_y, mu_x, mu_y, twiss_s = np.array([(tw.beta_x, tw.beta_y, tw.mux, tw.muy, tw.s) for tw in twiss]).T
bpm_beta_x, bpm_beta_y, bpm_mu_x, bpm_mu_y, bpm_twiss_s = np.array([(tw.beta_x, tw.beta_y, tw.mux, tw.muy, tw.s) for tw in twiss_bpms]).T

In [ ]:
fig = plt.figure(figsize=[20, 8])
ax = fig.add_subplot(111)
line_bx, = ax.plot(twiss_s, beta_x, 'b-', linewidth=.25)
ax.plot(bpm_twiss_s, bpm_beta_x, '.', color=line_bx.get_color())
line_by, = ax.plot(twiss_s, beta_y, 'r-', linewidth=.25)
ax.plot(bpm_twiss_s, bpm_beta_y, '.', color=line_by.get_color())
ax.set_ylabel(r'$\beta_x,\beta_y$' + ' [m]')
ax.set_xlabel('s [m]')

In [ ]:
fig = plt.figure(figsize=[20, 8])
ax = fig.add_subplot(111)
line_bx, = ax.plot(twiss_s, mu_x, 'b-', linewidth=.25)
ax.plot(bpm_twiss_s, bpm_mu_x, '.', color=line_bx.get_color())
line_by, = ax.plot(twiss_s, mu_y, 'r-', linewidth=.25)
ax.plot(bpm_twiss_s, bpm_mu_y, '.', color=line_by.get_color())
ax.set_ylabel(r'$\beta_x,\beta_y$' + ' [m]')
ax.set_xlabel('s [m]')

Closed orbit distortion

$$
    CO(s) = 
        \frac{\sqrt{\beta(s)}}{2 \sin (\pi Q)}
        \sum_i 
              \sqrt{\beta_i} \theta_i
              \cos(\pi Q - |\phi(s) - \phi_i|)
$$



In [ ]:
tw = twiss[1000]
tw

In [ ]:
tw.tau

In [ ]:
for ax in [ax1, ax2, ax3]:
    ax.clear()
    del ax


fig = plt.figure(figsize=[20,24])
ax1 = plt.subplot(311)
ax2 = plt.subplot(312)
ax3 = plt.subplot(313)

# ax3 = ax2.twinx()

# a polarity scale
p_scale = 1
for row in range(ref_data.x.shape[0]):
    # BPM measurement data
    # A line helping the eye
    dy_row = bpm_data.y[row, :]
    ds_row = bpm_data.s[row, :]
    
    dy_ref_row = ref_data.y[row, :] * 5

    # dx_ref_row = model_data[row, :] * model_to_bpm
    
    dy_m_row = bpm_data_m.y[row, :]
    
    plot_scale = prelative_current_scale[row]
    if plot_scale is None:
        plot_scale = 1/8
    plot_scale = p_scale * plot_scale
    
    pdy_row = dy_row * plot_scale
    pdy_m_row = dy_m_row * plot_scale
    pdy_ref_row = dy_ref_row * plot_scale
    
    # The measured data 
    line, = ax1.plot(ds_row, pdy_row, 'x')
    lcol = color=line.get_color()    
    ax1.plot(ds_row, pdy_row, '-', color=lcol, linewidth=.25)
    
    # The fitted ones 
    ax1.plot(offset_data.bpm.s,   pdy_ref_row, '+-', color=lcol, linewidth=.25)
  
    # Plotting the offset from the fit to all other data
    offset = dy_m_row - dy_ref_row
    # offset = dy_ref_row
    poffset = offset * plot_scale
    ax2.plot(offset_data.bpm.s, poffset, '+-', color=lcol, linewidth=.25)
    # relative difference only if significant measurement data
    # currently for bpm measurements above 0.05 mm
    bpm_min_measurement = 0.05 
    idx = np.absolute(pdy_m_row) > bpm_min_measurement
    
    offset_sel = offset[idx]
    dy_sel = offset_data.bpm.s[idx]
    dy_ref_row_sel = dy_ref_row[idx]
    scale = offset_sel / dy_ref_row_sel
    
    #ax3.plot(ds_sel, scale, '.--', color=lcol, linewidth=.25)

ax1.set_xlabel('ds [m]')
ax2.set_xlabel('ds [m]')
ax1.set_ylabel('y [mm]')
# ax2.set_ylabel('$\Delta$ y [mm]')
ax2.set_ylabel('$y_{ref}$ [mm]')
ax3.set_ylabel('ry')

In [ ]:
scipy.optimize.lsq_linear?